## multiclass variational classifier

In [1]:
import torch
import numpy as np
import pandas as pd
import pennylane as qml
from torch.autograd import Variable
import torch.optim as optim
from sklearn import datasets
import matplotlib.pyplot as plt
import pickle
import time
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
L_size = 512
data_path = 'train_fBm_dataset_shuffeld_L'+str(L_size)+'.csv'

num_layers = 18
total_iterations =600
num_classes = 10  
margin = 0.0375
batch_size = 10
lr_adam = 0.01
train_split = 0.75

# defs

In [3]:

def layer(W):
    for i in range(num_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    for j in range(num_qubits - 1):
        qml.CNOT(wires=[j, j + 1])
    if num_qubits >= 2:
       
        qml.CNOT(wires=[num_qubits - 1, 0])
        
###################################################################
def circuit(weights, feat=None):  #feat==feature
    qml.AmplitudeEmbedding(feat, range(num_qubits), pad_with=0.0, normalize=True)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))
########################################################################
def variational_classifier(q_circuit, params, feat):
    weights = params[0]
    bias = params[1]
    return q_circuit(weights, feat=feat) + bias

########################################################################
def multiclass_svm_loss(q_circuits, all_params, feature_vecs, true_labels):
    loss = 0
    num_samples = len(true_labels)
    for i, feature_vec in enumerate(feature_vecs):
     
        s_true = variational_classifier(q_circuits[int(true_labels[i])],
            (all_params[0][int(true_labels[i])], all_params[1][int(true_labels[i])]),feature_vec,)
        s_true = s_true.float()
        li = 0
        for j in range(num_classes):
            if j != int(true_labels[i]):
                s_j = variational_classifier(q_circuits[j], (all_params[0][j], all_params[1][j]), feature_vec)
                s_j = s_j.float()
                
                li += torch.max(torch.zeros(1).float(), s_j - s_true + margin)
        loss += li

    return loss / num_samples
########################################################################
def classify(q_circuits, all_params, feature_vecs, labels):
    predicted_labels = []
    for i, feature_vec in enumerate(feature_vecs):
        scores = np.zeros(num_classes)
        for c in range(num_classes):
            score = variational_classifier(
                q_circuits[c], (all_params[0][c], all_params[1][c]), feature_vec
            )
            scores[c] = float(score)
        pred_class = np.argmax(scores)
        predicted_labels.append(pred_class)
    return predicted_labels
########################################################################

def accuracy(labels, hard_predictions):
    correct = 0
    for l, p in zip(labels, hard_predictions):
        if torch.abs(l - p) < 1e-5:
            correct = correct + 1
    accuracy_amount= correct / labels.shape[0]
    return accuracy_amount
########################################################################
#load and normalize each input
def load_and_process_data(data_path):
    data = pd.read_csv(data_path)
    X = data.iloc[:,1:-1].values 
    Y = data.iloc[:,-1].values
    X=(X-np.mean(X,axis=1).reshape(-1,1))/(np.std(X,axis=1).reshape(-1,1))
    Y = torch.tensor(Y)
    X = torch.tensor(X)
    Y.shape,X.shape
    return X, Y
########################################################################
# Create a train and test split.
def split_data(feature_vecs, Y):
    num_data = len(Y)
    num_train = int(train_split * num_data)
    index = np.random.permutation(range(num_data))
    feat_vecs_train = feature_vecs[index[:num_train]]
    Y_train = Y[index[:num_train]]
    feat_vecs_test = feature_vecs[index[num_train:]]
    Y_test = Y[index[num_train:]]
    return feat_vecs_train, feat_vecs_test, Y_train, Y_test
########################################################################
def training(features, Y, rerun):
    num_data = Y.shape[0]
    feat_vecs_train, feat_vecs_test, Y_train, Y_test = split_data(features, Y)
    num_train = Y_train.shape[0]
    q_circuits = qnodes

    results_file=pd.read_excel('results.xlsx')

    if rerun==True:
       
        with open('best_params' + str(L_size) + '.pickle', 'rb') as f:
            loaded_params = pickle.load(f)
        loaded_weights, loaded_bias = loaded_params
        all_weights = [
        Variable(torch.tensor(w, dtype=torch.float32), requires_grad=True) 
        for w in loaded_weights
        ]
        all_bias = [
        Variable(torch.tensor(b, dtype=torch.float32), requires_grad=True)
        for b in loaded_bias
        ]

        # Initialize the optimizer
        optimizer = optim.Adam(all_weights + all_bias, lr=lr_adam)

        # Final params tuple
        params = (all_weights, all_bias)
        print("Num params: ", 3 * num_layers * num_qubits * 3 + 3)

    
    else:
        # Initialize the parameters
        all_weights = [
            Variable(0.1 * torch.randn(num_layers, num_qubits, 3), requires_grad=True)
            for i in range(num_classes)
        ]
        all_bias = [Variable(0.1 * torch.ones(1), requires_grad=True) for i in range(num_classes)]
        optimizer = optim.Adam(all_weights + all_bias, lr=lr_adam)
        params = (all_weights, all_bias)
        print("Num params: ", 3 * num_layers * num_qubits * 3 + 3)

    costs, train_acc, test_acc = [], [], []
    print("#############################################################################")
    for it in range(total_iterations):
        start=time.time()
        batch_index = np.random.randint(0, num_train, (batch_size,))
        feat_vecs_train_batch = feat_vecs_train[batch_index]
        Y_train_batch = Y_train[batch_index]

        optimizer.zero_grad()
        curr_cost = multiclass_svm_loss(q_circuits, params, feat_vecs_train_batch, Y_train_batch)
        curr_cost.backward()
        optimizer.step()

        # Compute predictions on train and validation set
        predictions_train = classify(q_circuits, params, feat_vecs_train, Y_train)
        predictions_test =classify(q_circuits, params, feat_vecs_test, Y_test)
        acc_train = accuracy(Y_train, predictions_train)
        acc_test = accuracy(Y_test, predictions_test)
                                                            
        new_row=pd.DataFrame({'Iter': [it + 1], 'Cost':[curr_cost.item()],'Acc train':[acc_train],'Acc test':[acc_test]})
        results_file=pd.concat([results_file,new_row],ignore_index=True)
        results_file.to_excel('results.xlsx',index=False)
        end=time.time()
        print(
            "Iter: {:5d} | time:{:0.5f} |Cost: {:0.7f} | Acc train: {:0.7f} | Acc test: {:0.7f} "
            "".format(it + 1,end-start ,curr_cost.item(), acc_train, acc_test)
        )
        with open('best_params' + str(L_size) + '.pickle', 'wb') as f:
            pickle.dump(params, f)
        
        
    

# Data Loading and Processing

In [4]:


data = pd.read_csv(data_path)
feature_size= len(data.iloc[[0]].values[0][1:-1])
print("length of ech time serie:",feature_size)
np.random.seed(0)
torch.manual_seed(0)
############################################################

# the number of the required qubits is calculated from the number of features
num_qubits = int(np.ceil(np.log2(feature_size)))

print("num_qubits :",num_qubits )

length of ech time serie: 512
num_qubits : 9


In [5]:
features, Y = load_and_process_data(data_path)
Y.shape,features.shape


(torch.Size([990]), torch.Size([990, 512]))

# training model  and plot the results

In [6]:
dev = qml.device("default.qubit", wires=num_qubits)
dev

<default.qubit device (wires=9) at 0x7e45f0089fd0>

In [7]:
qnodes = []
for iq in range(num_classes):
    qnode = qml.QNode(circuit, dev, interface="torch")  #The interface that will be used for classical backpropagation
    qnodes.append(qnode)                                #This affects the types of objects that can be passed to/returned from the QNode
q_circuits = qnodes
q_circuits

[<QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: device='<default.qubit device (wires=9) at 0x7e45f0089fd0>', interface='torch', diff_method='best'>,
 <QNode: d

In [8]:
# Initialize the parameters
print("Num params: ", 3 * num_layers * num_qubits * 3 + 3)

Num params:  1461


In [ ]:
#train modolFalse
rerun = False
start3 = time.time()
training(features, Y,rerun)
end3 = time.time()

print('total run time:',(end3 -start3 )/3600)

Num params:  1461
#############################################################################


/tmp/ipykernel_84956/679654816.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_file=pd.concat([results_file,new_row],ignore_index=True)


Iter:     1 | time:2654.94567 |Cost: 0.4286701 | Acc train: 0.1159030 | Acc test: 0.1209677 
Iter:     2 | time:2675.14546 |Cost: 0.4187289 | Acc train: 0.1172507 | Acc test: 0.1451613 
Iter:     3 | time:2652.61991 |Cost: 0.4008683 | Acc train: 0.1307278 | Acc test: 0.1491935 
Iter:     4 | time:2635.50237 |Cost: 0.4497766 | Acc train: 0.1778976 | Acc test: 0.1572581 
Iter:     5 | time:2652.13475 |Cost: 0.6820160 | Acc train: 0.1940701 | Acc test: 0.1653226 
Iter:     6 | time:2652.67882 |Cost: 0.4064651 | Acc train: 0.1954178 | Acc test: 0.1733871 
Iter:     7 | time:2652.89472 |Cost: 0.4200077 | Acc train: 0.2075472 | Acc test: 0.1814516 
Iter:     8 | time:2654.40010 |Cost: 0.3346764 | Acc train: 0.1415094 | Acc test: 0.1129032 
Iter:     9 | time:2656.93116 |Cost: 0.2865305 | Acc train: 0.1388140 | Acc test: 0.1048387 
Iter:    10 | time:2638.89285 |Cost: 0.3352032 | Acc train: 0.1617251 | Acc test: 0.1451613 
Iter:    11 | time:2650.97592 |Cost: 0.4365485 | Acc train: 0.1752022 